# Apriori Algorithm Implementation Assignment

### Objective:
You will implement the **Apriori algorithm** from scratch (i.e., without using any libraries like `mlxtend`) to find frequent itemsets and generate association rules.

### Dataset:
Use the [Online Retail Dataset](https://www.kaggle.com/datasets/vijayuv/onlineretail) from Kaggle. You can filter it for a specific country (e.g., `United Kingdom`) and time range to reduce size if needed.

---

## Step 1: Data Preprocessing

- Load the dataset
- Remove rows with missing values
- Filter out rows where `Quantity <= 0`
- Convert Data into Basket Format

👉 **Implement code below**

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv('./Datasets/OnlineRetail.csv', encoding='ISO-8859-1')

In [3]:
df = df.dropna(subset=['CustomerID', 'Description'])

df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

df = df[df['Country'] == 'United Kingdom']


df['Description'] = df['Description'].str.strip()
basket = (df.groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().fillna(0))

basket = basket.applymap(lambda x: 1 if x > 0 else 0)

transactions = []
for invoice, row in basket.iterrows():
    items = list(row[row == 1].index)
    if items:
        transactions.append(frozenset(items))

print(f"Total transactions (baskets): {len(transactions)}")
print(f"Total unique items (after filtering): {len(basket.columns)}")

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5592\551935786.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


Total transactions (baskets): 16649
Total unique items (after filtering): 3833


## Step 2: Implement Apriori Algorithm
Step-by-Step Procedure:
1. Generate Frequent 1-Itemsets
Count the frequency (support) of each individual item in the dataset.
Keep only those with support ≥ min_support.
→ Result is L1 (frequent 1-itemsets)
2. Iterative Candidate Generation (k = 2 to n)
While L(k-1) is not empty:
a. Candidate Generation

Generate candidate itemsets Ck of size k from L(k-1) using the Apriori property:
Any (k-itemset) is only frequent if all of its (k−1)-subsets are frequent.
b. Prune Candidates
Eliminate candidates that have any (k−1)-subset not in L(k-1).
c. Count Support
For each transaction, count how many times each candidate in Ck appears.
d. Generate Frequent Itemsets
Form Lk by keeping candidates from Ck that meet the min_support.
Repeat until Lk becomes empty.
Implement the following functions:
1. `get_frequent_itemsets(transactions, min_support)` - Returns frequent itemsets and their support
2. `generate_candidates(prev_frequent_itemsets, k)` - Generates candidate itemsets of length `k`
3. `calculate_support(transactions, candidates)` - Calculates the support count for each candidate

**Write reusable functions** for each part of the algorithm.

In [4]:
from itertools import combinations
from collections import defaultdict

In [5]:
def calculate_support(transactions, itemset):
    count = 0
    for t in transactions:
        if itemset.issubset(t):
            count += 1
    return count / len(transactions)

def get_single_item_counts(transactions):
    counts = defaultdict(int)
    for t in transactions:
        for item in t:
            counts[frozenset([item])] += 1
    return counts

def generate_candidates(freq_itemsets_prev, k):
    candidates = set()
    prev_list = list(freq_itemsets_prev)
    n = len(prev_list)
    for i in range(n):
        for j in range(i+1, n):
            a = prev_list[i]
            b = prev_list[j]
            union = a.union(b)
            if len(union) == k:
                candidates.add(frozenset(union))
    return candidates

def prune_candidates(candidates, freq_itemsets_prev, k):
    pruned = set()
    for c in candidates:
        all_subsets_frequent = True
        for subset in combinations(c, k-1):
            if frozenset(subset) not in freq_itemsets_prev:
                all_subsets_frequent = False
                break
        if all_subsets_frequent:
            pruned.add(c)
    return pruned

def get_frequent_itemsets(transactions, min_support=0.02):
 
    transactions = list(transactions)
    n_transactions = len(transactions)
    freq_itemsets = dict()
    
    single_counts = get_single_item_counts(transactions)
    L1 = dict()
    for itemset, cnt in single_counts.items():
        support = cnt / n_transactions
        if support >= min_support:
            L1[itemset] = support
    freq_itemsets[1] = L1
    
    k = 2
    prev_freq_itemsets = set(L1.keys())
    while prev_freq_itemsets:
        candidates = generate_candidates(prev_freq_itemsets, k)
        candidates = prune_candidates(candidates, prev_freq_itemsets, k)
        
        candidate_counts = defaultdict(int)
        for t in transactions:
            for c in candidates:
                if c.issubset(t):
                    candidate_counts[c] += 1
        Lk = dict()
        for c, cnt in candidate_counts.items():
            support = cnt / n_transactions
            if support >= min_support:
                Lk[c] = support
        
        if not Lk:
            break
        
        freq_itemsets[k] = Lk
        prev_freq_itemsets = set(Lk.keys())
        k += 1
    
    return freq_itemsets

min_support = 0.02  
freq_itemsets = get_frequent_itemsets(transactions, min_support=min_support)

total_itemsets = sum(len(v) for v in freq_itemsets.values())
print(f"Found {total_itemsets} frequent itemsets with min_support={min_support}")
for k in sorted(freq_itemsets):
    print(f"k={k}: {len(freq_itemsets[k])} itemsets")

Found 236 frequent itemsets with min_support=0.02
k=1: 200 itemsets
k=2: 35 itemsets
k=3: 1 itemsets


## Step 3: Generate Association Rules

- Use frequent itemsets to generate association rules
- For each rule `A => B`, calculate:
  - **Support**
  - **Confidence**
- Only return rules that meet a minimum confidence threshold (e.g., 0.5)

👉 **Implement rule generation function below**

In [6]:
from itertools import combinations

In [7]:

def generate_rules(freq_itemsets, min_confidence=0.5):
    
    support_lookup = dict()
    for k, d in freq_itemsets.items():
        for itemset, sup in d.items():
            support_lookup[itemset] = sup

    rules = []
    for k in freq_itemsets:
        if k < 2:
            continue
        for itemset in freq_itemsets[k]:
            items = list(itemset)
            for r in range(1, len(items)):
                for antecedent_tuple in combinations(items, r):
                    antecedent = frozenset(antecedent_tuple)
                    consequent = itemset - antecedent
                    if not consequent:
                        continue
                    support_itemset = support_lookup.get(itemset, 0)
                    support_antecedent = support_lookup.get(antecedent, calculate_support(transactions, antecedent))
                    support_consequent = support_lookup.get(consequent, calculate_support(transactions, consequent))
                    confidence = support_itemset / support_antecedent if support_antecedent > 0 else 0
                    lift = confidence / support_consequent if support_consequent > 0 else 0
                    if confidence >= min_confidence:
                        rules.append({
                            'antecedent': antecedent,
                            'consequent': consequent,
                            'support': support_itemset,
                            'confidence': confidence,
                            'lift': lift
                        })
    rules = sorted(rules, key=lambda x: (x['confidence'], x['lift']), reverse=True)
    return rules

min_confidence = 0.5
rules = generate_rules(freq_itemsets, min_confidence=min_confidence)
print(f"Generated {len(rules)} rules with min_confidence={min_confidence}")

Generated 30 rules with min_confidence=0.5


## Step 4: Output and Visualize

- Print top 10 frequent itemsets
- Print top 10 association rules (by confidence or lift)

👉 **Output results below**

In [8]:
from pprint import pprint

In [9]:

all_itemsets = []
for k, itm in freq_itemsets.items():
    for s, sup in itm.items():
        all_itemsets.append((s, sup))
all_itemsets_sorted = sorted(all_itemsets, key=lambda x: x[1], reverse=True)

print('\nTop 10 frequent itemsets:')
for itemset, sup in all_itemsets_sorted[:10]:
    print(f"Items (k={len(itemset)}): {set(itemset)}  --> support: {sup:.4f}")

print('\nTop 10 association rules (by confidence):')
for r in rules[:10]:
    print(f"Rule: {set(r['antecedent'])} -> {set(r['consequent'])} | support={r['support']:.4f}, confidence={r['confidence']:.4f}, lift={r['lift']:.4f}")


Top 10 frequent itemsets:
Items (k=1): {'WHITE HANGING HEART T-LIGHT HOLDER'}  --> support: 0.1132
Items (k=1): {'JUMBO BAG RED RETROSPOT'}  --> support: 0.0869
Items (k=1): {'REGENCY CAKESTAND 3 TIER'}  --> support: 0.0847
Items (k=1): {'ASSORTED COLOUR BIRD ORNAMENT'}  --> support: 0.0781
Items (k=1): {'PARTY BUNTING'}  --> support: 0.0775
Items (k=1): {'LUNCH BAG RED RETROSPOT'}  --> support: 0.0673
Items (k=1): {'SET OF 3 CAKE TINS PANTRY DESIGN'}  --> support: 0.0605
Items (k=1): {'LUNCH BAG  BLACK SKULL.'}  --> support: 0.0598
Items (k=1): {"PAPER CHAIN KIT 50'S CHRISTMAS"}  --> support: 0.0568
Items (k=1): {'NATURAL SLATE HEART CHALKBOARD'}  --> support: 0.0563

Top 10 association rules (by confidence):
Rule: {'ROSES REGENCY TEACUP AND SAUCER', 'PINK REGENCY TEACUP AND SAUCER'} -> {'GREEN REGENCY TEACUP AND SAUCER'} | support=0.0205, confidence=0.8903, lift=24.2210
Rule: {'GREEN REGENCY TEACUP AND SAUCER', 'PINK REGENCY TEACUP AND SAUCER'} -> {'ROSES REGENCY TEACUP AND SAUCER'}